# Partie 4.1 - Visualisations professionnelles avec Matplotlib

**ECF Data Engineer** - Analyse des consommations energetiques des batiments publics

Ce notebook produit 5 visualisations professionnelles a partir des donnees enrichies.  
Chaque figure respecte les bonnes pratiques : titre explicite, labels des axes avec unites, legende et annotations pertinentes.  
Les figures sont exportees en PNG a 300 dpi dans `../output/figures/`.

In [ ]:
# =============================================================================
# Imports et configuration
# =============================================================================
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
import pandas as pd
import numpy as np
from pathlib import Path

# Style professionnel
plt.style.use('seaborn-v0_8-whitegrid')

# Parametres globaux Matplotlib pour un rendu professionnel
plt.rcParams.update({
    'figure.figsize': (14, 7),
    'figure.dpi': 100,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight',
    'font.size': 12,
    'axes.titlesize': 16,
    'axes.labelsize': 13,
    'legend.fontsize': 11,
    'xtick.labelsize': 11,
    'ytick.labelsize': 11,
    'axes.titleweight': 'bold',
})

# Formatage francais pour les nombres
import locale
try:
    locale.setlocale(locale.LC_ALL, 'fr_FR.UTF-8')
except locale.Error:
    try:
        locale.setlocale(locale.LC_ALL, 'fr_FR')
    except locale.Error:
        print("Locale francaise non disponible, utilisation de la locale par defaut.")

# Repertoire de sortie pour les figures
FIGURES_DIR = Path('../output/figures')
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

# Chargement des donnees enrichies
df = pd.read_parquet('../output/consommations_enrichies.parquet')

print(f"Donnees chargees : {len(df):,} lignes x {df.shape[1]} colonnes")
print(f"Colonnes : {list(df.columns)}")
print(f"\nTypes d'energie : {df['type_energie'].unique()}")
print(f"Types de batiment : {df['type'].unique()}")
print(f"Classes energetiques : {sorted(df['classe_energetique'].unique())}")
print(f"Periode : {df['timestamp'].min()} -> {df['timestamp'].max()}")
df.head()

## Figure 1 : Evolution temporelle de la consommation totale par type d'energie

Graphique en lignes montrant l'evolution mensuelle de la consommation totale pour l'electricite, le gaz et l'eau sur la periode 2023-2024.  
L'eau est representee sur un axe secondaire car ses unites (m3) different de celles de l'electricite et du gaz (kWh).

In [ ]:
# =============================================================================
# Figure 1 : Evolution temporelle de la consommation totale par type d'energie
# =============================================================================

# Agreger les consommations mensuelles par type d'energie
df['annee_mois'] = df['timestamp'].dt.to_period('M')
evol_mensuelle = (
    df.groupby(['annee_mois', 'type_energie'])['consommation']
    .sum()
    .reset_index()
)
evol_mensuelle['date'] = evol_mensuelle['annee_mois'].dt.to_timestamp()

# Separer les trois types d'energie
elec = evol_mensuelle[evol_mensuelle['type_energie'] == 'electricite'].sort_values('date')
gaz = evol_mensuelle[evol_mensuelle['type_energie'] == 'gaz'].sort_values('date')
eau = evol_mensuelle[evol_mensuelle['type_energie'] == 'eau'].sort_values('date')

# Creation de la figure avec double axe Y
fig, ax1 = plt.subplots(figsize=(15, 7))

# Axe principal : electricite et gaz (kWh)
color_elec = '#E8A317'  # jaune-orange
color_gaz = '#2E86C1'   # bleu
color_eau = '#27AE60'   # vert

line_elec, = ax1.plot(elec['date'], elec['consommation'], '-o', color=color_elec,
                       linewidth=2.5, markersize=5, label='Electricite (kWh)', zorder=3)
line_gaz, = ax1.plot(gaz['date'], gaz['consommation'], '-s', color=color_gaz,
                      linewidth=2.5, markersize=5, label='Gaz (kWh)', zorder=3)

ax1.set_xlabel('Mois', fontweight='bold')
ax1.set_ylabel('Consommation (kWh)', fontweight='bold', color='#333333')
ax1.tick_params(axis='y', labelcolor='#333333')

# Axe secondaire : eau (m3)
ax2 = ax1.twinx()
line_eau, = ax2.plot(eau['date'], eau['consommation'], '-^', color=color_eau,
                      linewidth=2.5, markersize=5, label='Eau (m\u00b3)', zorder=3)
ax2.set_ylabel('Consommation d\'eau (m\u00b3)', fontweight='bold', color=color_eau)
ax2.tick_params(axis='y', labelcolor=color_eau)

# Formater les dates sur l'axe X
import matplotlib.dates as mdates
mois_fr = ['Jan', 'Fev', 'Mar', 'Avr', 'Mai', 'Jun',
           'Jul', 'Aou', 'Sep', 'Oct', 'Nov', 'Dec']
ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Annotation des pics de consommation
# Pic electricite
idx_pic_elec = elec['consommation'].idxmax()
pic_elec = elec.loc[idx_pic_elec]
ax1.annotate(f"Pic elec : {pic_elec['consommation']:,.0f} kWh",
             xy=(pic_elec['date'], pic_elec['consommation']),
             xytext=(30, 20), textcoords='offset points',
             fontsize=9, fontweight='bold', color=color_elec,
             arrowprops=dict(arrowstyle='->', color=color_elec, lw=1.5),
             bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor=color_elec, alpha=0.9))

# Pic gaz
idx_pic_gaz = gaz['consommation'].idxmax()
pic_gaz = gaz.loc[idx_pic_gaz]
ax1.annotate(f"Pic gaz : {pic_gaz['consommation']:,.0f} kWh",
             xy=(pic_gaz['date'], pic_gaz['consommation']),
             xytext=(30, -30), textcoords='offset points',
             fontsize=9, fontweight='bold', color=color_gaz,
             arrowprops=dict(arrowstyle='->', color=color_gaz, lw=1.5),
             bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor=color_gaz, alpha=0.9))

# Pic eau
idx_pic_eau = eau['consommation'].idxmax()
pic_eau = eau.loc[idx_pic_eau]
ax2.annotate(f"Pic eau : {pic_eau['consommation']:,.0f} m\u00b3",
             xy=(pic_eau['date'], pic_eau['consommation']),
             xytext=(-80, 20), textcoords='offset points',
             fontsize=9, fontweight='bold', color=color_eau,
             arrowprops=dict(arrowstyle='->', color=color_eau, lw=1.5),
             bbox=dict(boxstyle='round,pad=0.3', facecolor='white', edgecolor=color_eau, alpha=0.9))

# Legende combinee
lines = [line_elec, line_gaz, line_eau]
labels = [l.get_label() for l in lines]
ax1.legend(lines, labels, loc='upper left', frameon=True, fancybox=True, shadow=True)

# Titre
ax1.set_title('Evolution temporelle de la consommation totale par type d\'energie\n'
              'Batiments publics - Periode 2023-2024',
              fontsize=16, fontweight='bold', pad=15)

# Grille
ax1.grid(True, alpha=0.3, linestyle='--')
ax2.grid(False)

plt.tight_layout()
fig.savefig(FIGURES_DIR / 'fig01_evolution_temporelle.png', dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none')
plt.show()
print("Figure 1 sauvegardee : fig01_evolution_temporelle.png")

## Figure 2 : Distribution des consommations par type de batiment

Boxplot comparatif de la distribution des consommations pour les 5 types de batiments publics : ecole, mairie, gymnase, piscine, mediatheque.  
Les valeurs medianes sont annotees sur chaque boite.

In [ ]:
# =============================================================================
# Figure 2 : Distribution des consommations par type de batiment
# =============================================================================

# Ordre et couleurs pour les types de batiments
types_batiments = ['ecole', 'mairie', 'gymnase', 'piscine', 'mediatheque']
couleurs_bat = {
    'ecole': '#3498DB',       # bleu
    'mairie': '#E74C3C',      # rouge
    'gymnase': '#2ECC71',     # vert
    'piscine': '#9B59B6',     # violet
    'mediatheque': '#F39C12'  # orange
}

# Noms en francais avec majuscule pour l'affichage
labels_bat = {
    'ecole': 'Ecole',
    'mairie': 'Mairie',
    'gymnase': 'Gymnase',
    'piscine': 'Piscine',
    'mediatheque': 'Mediatheque'
}

# Preparer les donnees pour le boxplot
data_box = [df[df['type'] == t]['consommation'].dropna().values for t in types_batiments]

fig, ax = plt.subplots(figsize=(14, 7))

# Creer le boxplot
bp = ax.boxplot(
    data_box,
    labels=[labels_bat[t] for t in types_batiments],
    patch_artist=True,
    widths=0.6,
    showfliers=True,
    flierprops=dict(marker='o', markerfacecolor='grey', markersize=3, alpha=0.4),
    medianprops=dict(color='black', linewidth=2),
    whiskerprops=dict(color='#555555', linewidth=1.2),
    capprops=dict(color='#555555', linewidth=1.2)
)

# Appliquer les couleurs a chaque boite
for patch, t in zip(bp['boxes'], types_batiments):
    patch.set_facecolor(couleurs_bat[t])
    patch.set_alpha(0.7)
    patch.set_edgecolor('#333333')
    patch.set_linewidth(1.2)

# Annoter les valeurs medianes
for i, t in enumerate(types_batiments):
    mediane = np.median(data_box[i])
    ax.annotate(
        f'Med : {mediane:,.1f}',
        xy=(i + 1, mediane),
        xytext=(35, 5),
        textcoords='offset points',
        fontsize=10,
        fontweight='bold',
        color=couleurs_bat[t],
        bbox=dict(boxstyle='round,pad=0.3', facecolor='white',
                  edgecolor=couleurs_bat[t], alpha=0.9),
        arrowprops=dict(arrowstyle='->', color=couleurs_bat[t], lw=1.2)
    )

# Ajouter le nombre d'observations par type
for i, t in enumerate(types_batiments):
    n = len(data_box[i])
    ax.text(i + 1, ax.get_ylim()[0] + (ax.get_ylim()[1] - ax.get_ylim()[0]) * 0.02,
            f'n={n:,}', ha='center', va='bottom', fontsize=9, color='#666666', style='italic')

# Titre et labels
ax.set_title('Distribution des consommations energetiques par type de batiment\n'
             'Ensemble des types d\'energie confondus',
             fontsize=16, fontweight='bold', pad=15)
ax.set_xlabel('Type de batiment', fontweight='bold')
ax.set_ylabel('Consommation (kWh / m\u00b3)', fontweight='bold')

# Grille horizontale legere
ax.yaxis.grid(True, alpha=0.3, linestyle='--')
ax.xaxis.grid(False)

plt.tight_layout()
fig.savefig(FIGURES_DIR / 'fig02_distribution_batiments.png', dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none')
plt.show()
print("Figure 2 sauvegardee : fig02_distribution_batiments.png")

## Figure 3 : Heatmap consommation moyenne par heure et jour de semaine

Carte de chaleur representant la consommation moyenne selon l'heure de la journee (0-23) et le jour de la semaine (Lundi a Dimanche).  
Cette visualisation permet d'identifier les pics de consommation recurrents.

In [ ]:
# =============================================================================
# Figure 3 : Heatmap consommation moyenne par heure et jour de semaine
# =============================================================================

# Noms des jours en francais dans l'ordre Lundi -> Dimanche
jours_fr = ['Lundi', 'Mardi', 'Mercredi', 'Jeudi', 'Vendredi', 'Samedi', 'Dimanche']

# Mapping du jour_semaine (si numerique 0=Lundi) ou textuel
# On utilise la colonne 'heure' et 'jour_semaine' du dataframe
# Si jour_semaine est numerique (0-6), on l'utilise directement
# Sinon on le convertit
if df['jour_semaine'].dtype in ['int64', 'float64']:
    df['jour_num'] = df['jour_semaine'].astype(int)
else:
    jour_mapping = {j: i for i, j in enumerate(jours_fr)}
    # Gerer aussi les noms en minuscule
    jour_mapping.update({j.lower(): i for i, j in enumerate(jours_fr)})
    df['jour_num'] = df['jour_semaine'].map(jour_mapping)

# Creer le pivot table : moyenne de consommation par (heure, jour)
pivot_heatmap = df.pivot_table(
    values='consommation',
    index='heure',
    columns='jour_num',
    aggfunc='mean'
).sort_index()

# S'assurer que toutes les heures (0-23) et jours (0-6) sont presents
pivot_heatmap = pivot_heatmap.reindex(index=range(24), columns=range(7))

fig, ax = plt.subplots(figsize=(12, 9))

# Heatmap avec pcolormesh pour un meilleur controle
im = ax.pcolormesh(
    pivot_heatmap.columns.values,
    pivot_heatmap.index.values,
    pivot_heatmap.values,
    cmap='YlOrRd',
    shading='auto'
)

# Colorbar
cbar = fig.colorbar(im, ax=ax, pad=0.02)
cbar.set_label('Consommation moyenne (kWh / m\u00b3)', fontweight='bold', fontsize=12)

# Axes
ax.set_xticks(range(7))
ax.set_xticklabels(jours_fr, fontsize=11, fontweight='bold')
ax.set_yticks(range(0, 24, 1))
ax.set_yticklabels([f'{h:02d}h' for h in range(24)], fontsize=10)

ax.set_xlabel('Jour de la semaine', fontweight='bold', fontsize=13)
ax.set_ylabel('Heure de la journee', fontweight='bold', fontsize=13)

# Titre
ax.set_title('Consommation energetique moyenne par heure et jour de semaine\n'
             'Ensemble des batiments publics - Periode 2023-2024',
             fontsize=16, fontweight='bold', pad=15)

# Ajouter les valeurs dans les cellules pour les heures de pointe
seuil_annotation = pivot_heatmap.values[~np.isnan(pivot_heatmap.values)].mean() + \
                   pivot_heatmap.values[~np.isnan(pivot_heatmap.values)].std()
for h in range(24):
    for j in range(7):
        val = pivot_heatmap.iloc[h, j] if h < len(pivot_heatmap) and j < len(pivot_heatmap.columns) else np.nan
        if not np.isnan(val) and val >= seuil_annotation:
            ax.text(j, h, f'{val:.0f}',
                    ha='center', va='center', fontsize=7,
                    fontweight='bold', color='white')

# Inverser l'axe Y pour que 0h soit en haut
ax.invert_yaxis()

plt.tight_layout()
fig.savefig(FIGURES_DIR / 'fig03_heatmap_heure_jour.png', dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none')
plt.show()
print("Figure 3 sauvegardee : fig03_heatmap_heure_jour.png")

## Figure 4 : Temperature vs consommation de chauffage avec regression

Nuage de points representant la relation entre la temperature exterieure et la consommation de gaz (chauffage).  
Une droite de regression lineaire est ajoutee, avec l'equation et le coefficient de determination R\u00b2.  
Les points sont colores par saison.

In [ ]:
# =============================================================================
# Figure 4 : Temperature vs consommation de chauffage avec regression
# =============================================================================

# Filtrer les donnees de gaz (chauffage)
df_gaz = df[df['type_energie'] == 'gaz'].dropna(subset=['temperature_c', 'consommation']).copy()

# Couleurs par saison
couleurs_saison = {
    'hiver': '#2E86C1',       # bleu
    'printemps': '#27AE60',   # vert
    'ete': '#F39C12',         # orange
    'automne': '#E74C3C'      # rouge
}
# Gerer les variantes de noms de saison (majuscule/minuscule)
saisons_uniques = df_gaz['saison'].unique()
saison_map = {}
for s in saisons_uniques:
    s_lower = s.lower() if isinstance(s, str) else str(s).lower()
    if s_lower in couleurs_saison:
        saison_map[s] = s_lower

# Ordre d'affichage des saisons
ordre_saisons = ['hiver', 'printemps', 'ete', 'automne']
noms_saisons_fr = {
    'hiver': 'Hiver',
    'printemps': 'Printemps',
    'ete': 'Ete',
    'automne': 'Automne'
}

fig, ax = plt.subplots(figsize=(14, 8))

# Scatter par saison
for saison in ordre_saisons:
    # Trouver les lignes correspondant a cette saison
    mask = df_gaz['saison'].apply(
        lambda x: (x.lower() if isinstance(x, str) else str(x).lower()) == saison
    )
    subset = df_gaz[mask]
    if len(subset) > 0:
        ax.scatter(
            subset['temperature_c'],
            subset['consommation'],
            c=couleurs_saison[saison],
            alpha=0.35,
            s=20,
            label=noms_saisons_fr[saison],
            edgecolors='none'
        )

# Regression lineaire
x = df_gaz['temperature_c'].values
y = df_gaz['consommation'].values

# Calcul des coefficients
coeffs = np.polyfit(x, y, 1)
poly = np.poly1d(coeffs)

# Calcul du R2
y_pred = poly(x)
ss_res = np.sum((y - y_pred) ** 2)
ss_tot = np.sum((y - np.mean(y)) ** 2)
r2 = 1 - (ss_res / ss_tot)

# Tracer la droite de regression
x_range = np.linspace(x.min(), x.max(), 100)
ax.plot(x_range, poly(x_range), 'k-', linewidth=2.5, label='Regression lineaire', zorder=5)

# Afficher l'equation et R2
signe = '+' if coeffs[1] >= 0 else '-'
equation_text = (f'y = {coeffs[0]:.2f}x {signe} {abs(coeffs[1]):.2f}\n'
                 f'R\u00b2 = {r2:.4f}')
ax.text(
    0.98, 0.97, equation_text,
    transform=ax.transAxes, fontsize=13, fontweight='bold',
    verticalalignment='top', horizontalalignment='right',
    bbox=dict(boxstyle='round,pad=0.5', facecolor='white',
              edgecolor='#333333', alpha=0.95)
)

# Titre et labels
ax.set_title('Relation entre temperature exterieure et consommation de gaz (chauffage)\n'
             'Avec droite de regression lineaire',
             fontsize=16, fontweight='bold', pad=15)
ax.set_xlabel('Temperature exterieure (\u00b0C)', fontweight='bold')
ax.set_ylabel('Consommation de gaz (kWh)', fontweight='bold')

# Legende
ax.legend(loc='upper right', bbox_to_anchor=(0.98, 0.82),
          frameon=True, fancybox=True, shadow=True, fontsize=11)

# Grille
ax.grid(True, alpha=0.3, linestyle='--')

plt.tight_layout()
fig.savefig(FIGURES_DIR / 'fig04_temperature_chauffage.png', dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none')
plt.show()
print(f"Figure 4 sauvegardee : fig04_temperature_chauffage.png")
print(f"Equation : y = {coeffs[0]:.2f} * T + {coeffs[1]:.2f}")
print(f"R\u00b2 = {r2:.4f}")

## Figure 5 : Comparaison consommation par classe energetique

Diagramme en barres groupees comparant la consommation moyenne par m\u00b2 selon la classe DPE (A a G) et le type d'energie.  
Les barres d'erreur representent l'ecart-type. Les couleurs du fond suivent la convention DPE (vert pour A, rouge pour G).

In [ ]:
# =============================================================================
# Figure 5 : Comparaison consommation par classe energetique (DPE)
# =============================================================================

# Classes DPE ordonnees de A a G
classes_dpe = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

# Couleurs DPE conventionnelles (vert -> rouge)
couleurs_dpe = {
    'A': '#009B4D',  # vert fonce
    'B': '#50B848',  # vert clair
    'C': '#B5D334',  # jaune-vert
    'D': '#FFF200',  # jaune
    'E': '#F5A623',  # orange
    'F': '#E8491C',  # orange-rouge
    'G': '#D0021B'   # rouge
}

# Types d'energie
types_energie = ['electricite', 'gaz', 'eau']
labels_energie = {
    'electricite': 'Electricite (kWh/m\u00b2)',
    'gaz': 'Gaz (kWh/m\u00b2)',
    'eau': 'Eau (m\u00b3/m\u00b2)'
}
couleurs_energie = {
    'electricite': '#E8A317',
    'gaz': '#2E86C1',
    'eau': '#27AE60'
}
hatch_energie = {
    'electricite': '',
    'gaz': '//',
    'eau': '..'
}

# Filtrer uniquement les classes presentes dans les donnees
classes_presentes = [c for c in classes_dpe if c in df['classe_energetique'].unique()]

# Calculer moyenne et ecart-type de consommation_par_m2 par classe et type d'energie
stats_dpe = df.groupby(['classe_energetique', 'type_energie'])['consommation_par_m2'].agg(
    ['mean', 'std']
).reset_index()

fig, ax = plt.subplots(figsize=(15, 8))

n_classes = len(classes_presentes)
n_energies = len(types_energie)
largeur_barre = 0.25
x = np.arange(n_classes)

# Tracer les barres groupees par type d'energie
for i, energie in enumerate(types_energie):
    moyennes = []
    ecarts = []
    for classe in classes_presentes:
        subset = stats_dpe[
            (stats_dpe['classe_energetique'] == classe) &
            (stats_dpe['type_energie'] == energie)
        ]
        if len(subset) > 0:
            moyennes.append(subset['mean'].values[0])
            ecarts.append(subset['std'].values[0])
        else:
            moyennes.append(0)
            ecarts.append(0)

    offset = (i - (n_energies - 1) / 2) * largeur_barre
    barres = ax.bar(
        x + offset, moyennes, largeur_barre,
        yerr=ecarts, capsize=3,
        label=labels_energie[energie],
        color=couleurs_energie[energie],
        hatch=hatch_energie[energie],
        edgecolor='#333333', linewidth=0.8,
        alpha=0.85,
        error_kw=dict(elinewidth=1.2, ecolor='#555555', capthick=1)
    )

# Appliquer un fond colore DPE derriere chaque groupe de barres
ylim = ax.get_ylim()
for i, classe in enumerate(classes_presentes):
    ax.axvspan(
        i - 0.45, i + 0.45,
        alpha=0.08,
        color=couleurs_dpe.get(classe, '#CCCCCC'),
        zorder=0
    )

# Axes et labels
ax.set_xticks(x)
ax.set_xticklabels(
    [f'Classe {c}' for c in classes_presentes],
    fontsize=12, fontweight='bold'
)
ax.set_xlabel('Classe energetique (DPE)', fontweight='bold', fontsize=13)
ax.set_ylabel('Consommation moyenne par m\u00b2', fontweight='bold', fontsize=13)

# Titre
ax.set_title('Consommation moyenne par m\u00b2 selon la classe energetique DPE\n'
             'Par type d\'energie - Barres d\'erreur = ecart-type',
             fontsize=16, fontweight='bold', pad=15)

# Legende
ax.legend(loc='upper left', frameon=True, fancybox=True, shadow=True, fontsize=11)

# Grille horizontale
ax.yaxis.grid(True, alpha=0.3, linestyle='--')
ax.xaxis.grid(False)
ax.set_axisbelow(True)

# Ajouter une bande coloree DPE en bas (legende visuelle)
# Creer un axe supplementaire pour la bande de couleur DPE
ax2_dpe = fig.add_axes([0.125, -0.02, 0.775, 0.025])
for i, classe in enumerate(classes_presentes):
    ax2_dpe.barh(0, 1, left=i, color=couleurs_dpe.get(classe, '#CCCCCC'), edgecolor='white')
    ax2_dpe.text(i + 0.5, 0, classe, ha='center', va='center',
                 fontsize=10, fontweight='bold', color='white')
ax2_dpe.set_xlim(0, n_classes)
ax2_dpe.set_ylim(-0.5, 0.5)
ax2_dpe.axis('off')

plt.tight_layout()
fig.savefig(FIGURES_DIR / 'fig05_consommation_dpe.png', dpi=300, bbox_inches='tight',
            facecolor='white', edgecolor='none')
plt.show()
print("Figure 5 sauvegardee : fig05_consommation_dpe.png")

## Conclusion

Ce notebook a produit **5 visualisations professionnelles** avec Matplotlib :

| # | Figure | Fichier | Description |
|---|--------|---------|-------------|
| 1 | Evolution temporelle | `fig01_evolution_temporelle.png` | Courbes mensuelles par type d'energie avec double axe Y |
| 2 | Distribution par batiment | `fig02_distribution_batiments.png` | Boxplot comparatif des 5 types de batiments |
| 3 | Heatmap heure x jour | `fig03_heatmap_heure_jour.png` | Carte de chaleur des consommations moyennes |
| 4 | Temperature vs chauffage | `fig04_temperature_chauffage.png` | Nuage de points avec regression lineaire et R\u00b2 |
| 5 | Consommation par DPE | `fig05_consommation_dpe.png` | Barres groupees par classe energetique et type d'energie |

**Bonnes pratiques respectees pour chaque figure :**
- Titre explicite et descriptif
- Labels des axes avec unites
- Legende claire
- Annotations pertinentes (pics, medianes, equation, R\u00b2)
- Mise en page soignee (`tight_layout`)
- Export en haute resolution (300 dpi)
- Textes en francais

Toutes les figures sont sauvegardees dans le repertoire `../output/figures/`.